In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/train_data.csv')
test = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/test_data.csv')
train_labels=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/train_labels.csv')
fitness=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/fitness_values.csv')


In [2]:
list=(['patient_id', 'event_name','specialty','plan_type'])
    
for x in train.columns:
        if x in list:
            train[x]=train[x].astype(np.object)
for x in test.columns:
        if x in list:
            test[x]=test[x].astype(np.object)

In [3]:
print(train.shape)
train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

print(train.shape)
print(test.shape)
test.head()

(14446880, 6)
(7171012, 6)
(3083639, 6)


,patient_id,event_name,event_time,specialty,plan_type,patient_payment
0,patient_2,event_1,164,spec_1,plan_2,0.0
1,patient_3,event_1,420,spec_1,plan_2,0.0
2,patient_5,event_1,623,spec_1,plan_2,0.0
3,patient_8,event_1,162,spec_1,plan_1,0.0
4,patient_14,event_1,243,spec_1,plan_2,0.0


In [4]:
train['flag']='train'
test['flag']='test'

train.head()

,patient_id,event_name,event_time,specialty,plan_type,patient_payment,flag
0,patient_1,event_1,47,spec_1,plan_1,0.0,train
1,patient_4,event_1,727,spec_1,plan_2,0.0,train
2,patient_6,event_1,738,spec_1,plan_1,0.0,train
3,patient_7,event_1,923,spec_1,plan_1,0.0,train
4,patient_9,event_1,49,spec_1,plan_1,0.0,train


In [5]:
combo=train.append(test,sort=False)
combo.shape
combo.head()
combo['patient_id'].value_counts()
combo.shape


(10254651, 7)

In [43]:
combo_pivot = combo.pivot_table(index=['patient_id','flag'], columns='event_name', values=('event_time','patient_payment'), aggfunc=[np.min,np.max,np.mean,np.sum])
combo_pivot.drop([['mean','patient_payment'],['amin','patient_payment'],['amax','patient_payment'],['sum','event_time']],axis=1,inplace=True)
combo_pivot.shape


combo_pivot.reset_index(inplace=True)
#combo_pivot.set_index('patient_id',inplace=True)
combo_pivot

patient_id   flag       amin                               \
                                 event_time                                
event_name                          event_1 event_10 event_100 event_101   
0               patient_1  train       38.0    382.0       NaN     428.0   
1              patient_10  train      499.0     44.0       NaN     499.0   
2             patient_100  train      308.0      NaN       NaN       NaN   
3            patient_1000  train       62.0      NaN       NaN     116.0   
4           patient_10000  train        NaN    527.0       NaN       NaN   
...                   ...    ...        ...      ...       ...       ...   
23826        patient_9995  train        NaN      NaN       NaN       NaN   
23827        patient_9996  train        NaN    128.0     131.0      99.0   
23828        patient_9997  train        NaN      NaN       NaN       NaN   
23829        patient_9998   test       30.0      NaN       NaN       NaN   
23830        patient_9999  train        NaN      NaN       NaN     506.0   

                                                    ...             sum  \
                                                    ... patient_payment   
event_name event_102 event_103 event_104 event_105  ...        event_90   
0                NaN       NaN     396.0      47.0  ...             0.0   
1                NaN       NaN      93.0     128.0  ...             NaN   
2                NaN       NaN       NaN      47.0  ...             NaN   
3                NaN       NaN       NaN      24.0  ...             0.0   
4                NaN       NaN     416.0      52.0  ...             NaN   
...              ...       ...       ...       ...  ...             ...   
23826            NaN       NaN       NaN     104.0  ...           130.0   
23827            NaN       NaN       NaN      93.0  ...             NaN   
23828            NaN       NaN       NaN     497.0  ...             0.0   
23829            NaN       NaN       NaN       NaN  ...             NaN   
23830            NaN       NaN       NaN       2.0  ...             NaN   

                                                                           \
                                                                            
event_name event_91 event_92 event_93 event_94 event_95 event_96 event_97   
0               NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1               NaN      NaN    91.25      NaN      NaN      NaN      NaN   
2               NaN      NaN      NaN      NaN      NaN      0.0      NaN   
3               NaN      NaN      NaN      NaN      NaN      9.9      NaN   
4               NaN      NaN      NaN      NaN      NaN      NaN     9.85   
...             ...      ...      ...      ...      ...      ...      ...   
23826           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
23827           NaN      NaN      NaN      NaN      NaN      0.0      NaN   
23828           NaN      NaN      NaN      NaN      NaN      NaN    50.00   
23829           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
23830           NaN      NaN      NaN      NaN     75.0      NaN      NaN   

                              
                              
event_name event_98 event_99  
0               NaN     0.00  
1               NaN  1002.69  
2               NaN     0.00  
3               NaN      NaN  
4               0.0     0.00  
...             ...      ...  
23826           NaN    15.00  
23827           NaN     0.00  
23828           NaN      NaN  
23829           NaN      NaN  
23830           0.0      NaN  

[23831 rows x 2082 columns]

In [7]:
combo_count_pos_pays=combo[combo['patient_payment'] > 0].groupby('patient_id')['patient_payment'].count().sort_values(ascending=False).reset_index()
combo_count_number_pays=combo.groupby('patient_id')['patient_payment'].count().sort_values(ascending=False).reset_index()

combo_counts_unique_event=combo.groupby('patient_id')['event_name'].nunique().sort_values(ascending=False).reset_index()
combo_counts_unique_spec=combo.groupby('patient_id')['specialty'].nunique().sort_values(ascending=False).reset_index()
combo_counts_unique_plan=combo.groupby('patient_id')['plan_type'].nunique().sort_values(ascending=False).reset_index()

combo_max_pay=combo.groupby('patient_id')['patient_payment'].max().sort_values(ascending=False).reset_index()

combo_max_time=combo.groupby('patient_id')['event_time'].max().sort_values(ascending=False).reset_index()
combo_min_time=combo.groupby('patient_id')['event_time'].min().sort_values(ascending=False).reset_index()
combo_mean_time=combo.groupby('patient_id')['event_time'].mean().sort_values(ascending=False).reset_index()


dfs=([combo_count_pos_pays,combo_count_number_pays,combo_counts_unique_event,combo_counts_unique_spec,combo_counts_unique_plan,combo_max_pay,combo_max_time,combo_min_time,combo_mean_time])

from functools import reduce
combo_counts = reduce(lambda  left,right: pd.merge(left,right,on=['patient_id'],
                                            how='outer'), dfs).fillna(0) 


combo_counts.rename(columns={'patient_payment_x':'count_pos_pays','patient_payment_y':'count_rows','event_name':'count_unique_events','specialty':'count_unique_specs','plan_type':'count_unique_plans','patient_payment':'combo_max_pay','event_time_x':'time_max','event_time_y':'time_min','event_time':'time_mean'}, inplace=True)
combo_counts['paid_rows_proportion']=combo_counts['count_pos_pays']/combo_counts['count_rows']




In [46]:
#combo_categ_sort = combo.groupby(['patient_id', 'event_name'], as_index=False)['specialty','plan_type'].count()\
#        .sort_values(['specialty','plan_type'], ascending=False)

combo_categ_sort_spec=combo.groupby(['patient_id'])['specialty'].max().reset_index()
combo_categ_sort_plan=combo.groupby(['patient_id'])['plan_type'].max().reset_index()




In [47]:
#combo_categ_sort.drop_duplicates(keep=False,inplace=True)
combo_categ_sort=pd.merge(combo_categ_sort_spec,combo_categ_sort_plan,left_on=['patient_id'],right_on=['patient_id'])
#combo_categ_sort.set_index('patient_id',inplace=True)
#data.set_index("First Name", inplace = True) 

combo_categ_sort
#combo_categ_sort_spec
#combo_dump= combo[combo['patient_id']=='patient_1']
#combo_dump.append(combo[combo['patient_id']=='patient_1'])
#combo_dump.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/highest_freq_test',index=False)
#combo_categ_sort
#print(combo_dump[combo_dump['event_name']=='event_68'])

,patient_id,specialty,plan_type
0,patient_1,spec_9,plan_5
1,patient_10,spec_9,plan_5
2,patient_100,spec_6,plan_5
3,patient_1000,spec_9,plan_5
4,patient_10000,spec_6,plan_5
...,...,...,...
23826,patient_9995,spec_9,plan_5
23827,patient_9996,spec_9,plan_6
23828,patient_9997,spec_62,plan_6
23829,patient_9998,spec_51,plan_6


In [48]:
#combo_pivot.reset_index(inplace=True)
combo_pivot.head()

patient_id   flag       amin                               \
                                 event_time                                
event_name                          event_1 event_10 event_100 event_101   
0               patient_1  train       38.0    382.0       NaN     428.0   
1              patient_10  train      499.0     44.0       NaN     499.0   
2             patient_100  train      308.0      NaN       NaN       NaN   
3            patient_1000  train       62.0      NaN       NaN     116.0   
4           patient_10000  train        NaN    527.0       NaN       NaN   

                                                    ...             sum  \
                                                    ... patient_payment   
event_name event_102 event_103 event_104 event_105  ...        event_90   
0                NaN       NaN     396.0      47.0  ...             0.0   
1                NaN       NaN      93.0     128.0  ...             NaN   
2                NaN       NaN       NaN      47.0  ...             NaN   
3                NaN       NaN       NaN      24.0  ...             0.0   
4                NaN       NaN     416.0      52.0  ...             NaN   

                                                                           \
                                                                            
event_name event_91 event_92 event_93 event_94 event_95 event_96 event_97   
0               NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1               NaN      NaN    91.25      NaN      NaN      NaN      NaN   
2               NaN      NaN      NaN      NaN      NaN      0.0      NaN   
3               NaN      NaN      NaN      NaN      NaN      9.9      NaN   
4               NaN      NaN      NaN      NaN      NaN      NaN     9.85   

                              
                              
event_name event_98 event_99  
0               NaN     0.00  
1               NaN  1002.69  
2               NaN     0.00  
3               NaN      NaN  
4               0.0     0.00  

[5 rows x 2082 columns]

In [49]:
#combo_categ_sort.reset_index(inplace=True)
combo_categ_sort.head()

,patient_id,specialty,plan_type
0,patient_1,spec_9,plan_5
1,patient_10,spec_9,plan_5
2,patient_100,spec_6,plan_5
3,patient_1000,spec_9,plan_5
4,patient_10000,spec_6,plan_5


In [50]:
combo_counts.head()

,patient_id,count_pos_pays,count_rows,count_unique_events,count_unique_specs,count_unique_plans,combo_max_pay,time_max,time_min,time_mean,paid_rows_proportion
0,patient_1075,1390.0,3335,118,6,3,322.157143,1079,1,679.029385,0.416792
1,patient_55,1311.0,2835,129,19,3,250.000000,1075,7,541.945679,0.462434
2,patient_2684,1251.0,2222,96,7,3,250.000000,1074,1,439.319082,0.563006
3,patient_4900,1232.0,2243,133,16,3,250.000000,1080,3,553.317432,0.549264
4,patient_3197,1216.0,2066,125,14,3,250.000000,1078,1,462.323814,0.588577


In [51]:
combo_pivot.columns = [' '.join(col).strip() for col in combo_pivot.columns.values]

combo_pivot=pd.merge(combo_pivot,combo_categ_sort,left_on=['patient_id'],right_on=['patient_id'])
combo_pivot=pd.merge(combo_pivot,combo_counts,left_on=['patient_id'],right_on=['patient_id'])

#a = np.intersect1d(combo_pivot.columns, combo_categ_sort.columns)
#print (a)
#['B' 'C']

#a = df2.columns.intersection(df1.columns)
#print (a)
#Index(['B', 'C'], dtype='object')
#Alternative syntax for the latter option:

#print(combo_pivot.columns & combo_pivot_categ.columns)

#test.columns
print(combo_pivot.head(100))
#print(combo_pivot.columns.to_string())

       patient_id   flag  amin event_time event_1  amin event_time event_10  \
0       patient_1  train                     38.0                     382.0   
1      patient_10  train                    499.0                      44.0   
2     patient_100  train                    308.0                       NaN   
3    patient_1000  train                     62.0                       NaN   
4   patient_10000  train                      NaN                     527.0   
..            ...    ...                      ...                       ...   
95  patient_10083  train                      NaN                       NaN   
96  patient_10084  train                      NaN                       NaN   
97  patient_10085   test                    278.0                       NaN   
98  patient_10086   test                      NaN                       NaN   
99  patient_10087  train                      NaN                       NaN   

    amin event_time event_100  amin event_time even

In [52]:
y = combo_pivot[combo_pivot.columns[-15:]].copy()
y

,sum patient_payment event_97,sum patient_payment event_98,sum patient_payment event_99,specialty,plan_type,count_pos_pays,count_rows,count_unique_events,count_unique_specs,count_unique_plans,combo_max_pay,time_max,time_min,time_mean,paid_rows_proportion
0,NaN,NaN,0.00,spec_9,plan_5,77.0,540,75,9,3,20.100000,1079,12,500.720370,0.142593
1,NaN,NaN,1002.69,spec_9,plan_5,306.0,807,98,17,3,275.000000,1079,2,496.765799,0.379182
2,NaN,NaN,0.00,spec_6,plan_5,85.0,799,67,8,3,222.000000,1077,6,614.191489,0.106383
3,NaN,NaN,NaN,spec_9,plan_5,326.0,660,88,14,3,250.000000,1079,9,474.393939,0.493939
4,9.85,0.0,0.00,spec_6,plan_5,67.0,468,71,12,3,30.000000,1079,11,602.940171,0.143162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23826,NaN,NaN,15.00,spec_9,plan_5,464.0,599,72,13,3,136.620000,1080,7,604.040067,0.774624
23827,NaN,NaN,0.00,spec_9,plan_6,78.0,791,87,8,3,107.142857,1080,16,417.711757,0.098609
23828,50.00,NaN,NaN,spec_62,plan_6,129.0,447,64,11,3,321.428571,1080,3,537.029083,0.288591
23829,NaN,NaN,NaN,spec_51,plan_6,64.0,129,38,8,3,37.500000,1073,8,326.751938,0.496124


In [54]:
#non_dummy_cols = ['patient_id','flag','C'] 
# Takes all 47 other columns
#dummy_cols = list(set(df.columns) - set(non_dummy_cols))
#combo_pivot_ids=combo_pivot['patient_id']
#combo_pivot.drop(['patient_id_y'],axis=1,inplace=True)
combo_pivot = pd.get_dummies(combo_pivot, columns=['specialty','plan_type'])
print(combo_pivot_ids)

0            patient_1
1           patient_10
2          patient_100
3         patient_1000
4        patient_10000
             ...      
23826     patient_9995
23827     patient_9996
23828     patient_9997
23829     patient_9998
23830     patient_9999
Name: patient_id, Length: 23831, dtype: object


In [55]:
combo_pivot.shape

(23831, 2186)

In [62]:
train=combo_pivot[combo_pivot['flag']=='train']
test=combo_pivot[combo_pivot['flag']=='test']

print(test.shape)
print(train.shape)


(7148, 2186)
(16683, 2186)


In [12]:
#train.columns = [' '.join(col).strip() for col in train.columns.values]
#test.columns=[' '.join(col).strip() for col in test.columns.values]

In [63]:
train=pd.merge(train,train_labels,left_on='patient_id',right_on='patient_id')
#train['outcome_flag']=train.outcome_flag.apply(lambda x: 'Yes' if x == 1 else 'No')
train.head()

,patient_id,flag,amin event_time event_1,amin event_time event_10,amin event_time event_100,amin event_time event_101,amin event_time event_102,amin event_time event_103,amin event_time event_104,amin event_time event_105,...,specialty_spec_97,specialty_spec_98,specialty_spec_99,plan_type_plan_1,plan_type_plan_2,plan_type_plan_3,plan_type_plan_4,plan_type_plan_5,plan_type_plan_6,outcome_flag
0,patient_1,train,38.0,382.0,NaN,428.0,NaN,NaN,396.0,47.0,...,0,0,0,0,0,0,0,1,0,0
1,patient_10,train,499.0,44.0,NaN,499.0,NaN,NaN,93.0,128.0,...,0,0,0,0,0,0,0,1,0,1
2,patient_100,train,308.0,NaN,NaN,NaN,NaN,NaN,NaN,47.0,...,0,0,0,0,0,0,0,1,0,0
3,patient_1000,train,62.0,NaN,NaN,116.0,NaN,NaN,NaN,24.0,...,0,0,0,0,0,0,0,1,0,0
4,patient_10000,train,NaN,527.0,NaN,NaN,NaN,NaN,416.0,52.0,...,0,0,0,0,0,0,0,1,0,0


In [64]:
print(test.shape)
print(train.shape)
train.drop(['patient_id'],axis=1,inplace=True)
test_ids=test['patient_id']
test.drop(['patient_id'],axis=1,inplace=True)
print(test.shape)
print(train.shape)

(7148, 2186)
(16683, 2187)
(7148, 2185)
(16683, 2186)


In [65]:
test_ids

6        patient_10002
7        patient_10003
8        patient_10004
10       patient_10006
16       patient_10011
             ...      
23817     patient_9987
23818     patient_9988
23819     patient_9989
23822     patient_9991
23829     patient_9998
Name: patient_id, Length: 7148, dtype: object

In [66]:
train.drop(['flag'],axis=1,inplace=True)
test.drop(['flag'],axis=1,inplace=True)


In [ ]:
#train.event_name_order.nunique()
#, values =['patient_id','event_name', 'event_time', 'specialty', 'plan_type',
#       'patient_payment', 'outcome_flag', 'event_order']

#train_trial=train
#print(train_trial.shape)
#train_trial.drop_duplicates(keep=False,inplace=True)
#print(train_trial.shape)

#train_trial=train_trial[['patient_id','event_name_order','event_time']]
#train_trial['key']=train_trial['patient_id']+train_trial['event_name_order']+train_trial['event_time'].map(str)

#train_trial.drop_duplicates(keep=False,inplace=True)
#train_trial.shape

In [17]:
#train.drop('event_name',axis=1,inplace=True)
train.head()

,mean event_time event_1,mean event_time event_10,mean event_time event_100,mean event_time event_101,mean event_time event_102,mean event_time event_103,mean event_time event_104,mean event_time event_105,mean event_time event_106,mean event_time event_107,...,specialty_spec_97,specialty_spec_98,specialty_spec_99,plan_type_plan_1,plan_type_plan_2,plan_type_plan_3,plan_type_plan_4,plan_type_plan_5,plan_type_plan_6,outcome_flag
0,45.000000,418.000000,NaN,542.0,NaN,NaN,594.500,524.800000,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
1,732.800000,232.571429,NaN,580.0,NaN,NaN,312.125,287.000000,NaN,259.384615,...,0,0,0,0,0,0,0,1,0,1
2,308.000000,NaN,NaN,NaN,NaN,NaN,NaN,799.454545,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
3,384.705882,NaN,NaN,129.5,NaN,NaN,NaN,140.000000,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
4,NaN,545.666667,NaN,NaN,NaN,NaN,928.000,566.333333,186.0,NaN,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#train=train.unstack()
#train_pivot=train_trial.pivot(index ='patient_id', columns ='event_name_order') 
#train_pivot = train.pivot_table(index='patient_id', columns='event_name_order', values=('event_time','specialty','plan_type','patient_payment'), aggfunc=np.min)

In [18]:
#train.describe(include='all')
test.shape

(7148, 1134)

In [ ]:
#train_pivot.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/pivot',index=False)

In [67]:
#Alternate code for grodsearch

# A parameter grid for XGBoost
params = {
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.4, 0.8],
        'max_depth': [ 5, 8],
        'learning_rate':[0.01,0.1,0.3],
        'n_estimators':[30,50,80]
        }

In [68]:
#Alternate code for gridsearch/tuning

Xd,yd=train.drop('outcome_flag',axis=1),train.outcome_flag 
from sklearn.metrics import mean_squared_error
import xgboost as xgb
xg = xgb.XGBRegressor(eval_metric='auc',verbose=100, nthread=4)          

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=xg, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(Xd,yd), verbose=100 )
grid.fit(Xd, yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/Tuning_output_XGBReg_pivot_aggr_added_features_v3', index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to new file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to new file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11121,)

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.2min
Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  1.2min

Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11121,), dtype=int32).


[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  3.1min
Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11121,), dtype=int32).
Pickling array (shape=(5562,), dtype=int32).
[Parallel(n_jobs

[Parallel(n_jobs=4)]: Done  23 tasks      | elapsed:  6.0min
Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11121,), dtype=int32).
Pickling array (shape=(5562,), dtype=int32).
[Parallel(n_jobs

[Parallel(n_jobs=4)]: Done  32 tasks      | elapsed:  7.8min
Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11121,), dtype=int32).
Pickling array (shape=(5562,), dtype=int32).
[Parallel(n_jobs

...

[Parallel(n_jobs=4)]: Done 301 tasks      | elapsed: 108.0min
Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11123,), dtype=int32).
Pickling array (shape=(5560,), dtype=int32).
[Parallel(n_job

[Parallel(n_jobs=4)]: Done 310 tasks      | elapsed: 109.7min
Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11123,), dtype=int32).
Pickling array (shape=(5560,), dtype=int32).
[Parallel(n_job

Pickling array (shape=(2184,), dtype=object).
Pickling array (shape=(16683,), dtype=int64).
Memmapping (shape=(2084, 16683), dtype=float64) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-056de471646d42039fc24c84a8c4d55b.pkl
Pickling array (shape=(6, 16683), dtype=int64).
Memmapping (shape=(94, 16683), dtype=uint8) to old file C:\Users\np8022\AppData\Local\Temp\joblib_memmapping_folder_6392_2223514904\6392-2000793322696-00e96e07ae314ec08ad5a571d45f9968.pkl
Pickling array (shape=(2084,), dtype=object).
Pickling array (shape=(6,), dtype=object).
Pickling array (shape=(94,), dtype=object).
Pickling array (shape=(2084,), dtype=int64).
Pickling array (shape=(6,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(16683,), dtype=int64).
Pickling array (shape=(11123,), dtype=int32).
Pickling array (shape=(5560,), dtype=int32).
[Parallel(n_jobs=4)]: Done 321 out of 324 | elapsed: 113.3min remaining:  1.1

C:\Users\np8022\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\np8022\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[00:43:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [69]:
#XGBoost code,using tuned models to predict

X,y=train.drop('outcome_flag',axis=1),train.outcome_flag 

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

import xgboost as xgb
xg_reg = xgb.XGBRegressor(eval_metric='auc',learning_rate = 0.1,
                max_depth = 5,colsample_bytree=0.8, subsample=1, n_estimators = 80,verbose=50)

err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
#fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1994)

#for train_index, test_index in fold.split(X,y):
#    X_train, X_test = Xd.iloc[train_index], Xd.iloc[test_index]
#    y_train, y_test = yd[train_index], yd[test_index]
print('start')
xg_reg.fit(X,y)
#    print('pred...')
#    p=xg_reg.predict(X_test)
#    print("err: ",100*np.sqrt(mean_squared_error(y_test,p)))
#    err.append(100*np.sqrt(mean_squared_error(y_test,p)))
pred = xg_reg.predict(test)
y_pred_tot.append(pred)

start
[00:48:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [70]:
# y_pred=rf.predict(dftest_dum.drop('id',axis=1))
print(np.mean(err,0))
y_pred=np.mean(y_pred_tot,0)
print(y_pred)

s=pd.DataFrame({'patient_id':test_ids,'outcome_flag':y_pred})
s.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/XGB_pivot_aggr_tuned_added_features',index=False)
s.head()

nan
[0.0598537  0.2718075  0.04296419 ... 0.31302577 0.09777674 0.13730162]


C:\Users\np8022\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\np8022\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,patient_id,outcome_flag
6,patient_10002,0.059854
7,patient_10003,0.271807
8,patient_10004,0.042964
10,patient_10006,0.078212
16,patient_10011,0.022148


In [62]:
test_d = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/test_data.csv')

test_p['patient_id']=test_d['patient_id']
#test_p.drop_duplicates(keep=False,inplace=True)
#test_p.drop_duplicates(keep=False,inplace=True)

test_p.head()

patient_id    0              patient_2
1              patien...
Name: patient_id, dtype: object

In [50]:
print(test_d[test_d['patient_id']=='patient_22686'])

            patient_id event_name  event_time specialty plan_type  \
241974   patient_22686   event_22         805    spec_1    plan_2   
382452   patient_22686   event_37         805    spec_1    plan_2   
540472   patient_22686   event_37         805    spec_1    plan_2   
750247   patient_22686   event_22         805    spec_1    plan_2   
797473   patient_22686   event_37         805    spec_1    plan_2   
839707   patient_22686  event_104         805    spec_1    plan_2   
904823   patient_22686   event_98         805    spec_1    plan_2   
972863   patient_22686   event_22         805    spec_1    plan_2   
1049810  patient_22686   event_22         805    spec_1    plan_2   
1144833  patient_22686   event_37         805    spec_1    plan_2   
1150853  patient_22686  event_104         805    spec_1    plan_2   
1310404  patient_22686   event_98         805    spec_1    plan_2   
1488764  patient_22686   event_98         805    spec_1    plan_2   
1570966  patient_22686  event_104 